## Cognitive Services

### Configuración Inicial

Primero, configuraremos todas las urls y las claves necesarias para ejecutar los distintos servicios cognitivos.


In [ ]:
cs_key = "911e9745a6604f088c56ea1b9cb02c87"
cs_base_url = "https://westeurope.api.cognitive.microsoft.com/"
bs_account_name = "eatraining3939401398"
translate_url = "https://api.cognitive.microsofttranslator.com/translate?api-version=3.0"
translate_key = "aeac4c9876ee48b1b7c9adde6f5537fd"

In [ ]:
import requests
import os
import matplotlib.pyplot as plt
from PIL import Image
from matplotlib.patches import Rectangle

class CognitiveServices():
    def __init__(self, base_url, api_key, translate_url):
        self.__api_key = api_key
        self.__base_url = base_url
        self.__api_version = "v2.0"
        self.__translate_url = translate_url
        self.__init_config()
        
    def __init_config(self):
        
        self.__headers = {'Ocp-Apim-Subscription-Key': self.__api_key }
        self.__custom_vision_url = f"{self.__base_url}vision/{self.__api_version}/"
        self.__text_anaytics_url = f"{self.__base_url}text/analytics/{self.__api_version}/"
    
    def computer_vision_api(self, blob_url):
        vision_analyze_url = self.__custom_vision_url + "analyze"
        params   = {'visualFeatures': 'Faces,Tags,Categories,Description,Color'}
        data     = {'url': blob_url}
        response = requests.post(vision_analyze_url, headers= self.__headers, params=params, json=data)
        response.raise_for_status()
        analysis = response.json()
        return analysis
    
    def ocr(self, blob_url, filename, draw_image=False):
        ocr_url = self.__custom_vision_url + "ocr"
        params   = {'language': 'unk', 'detectOrientation ': 'true'}
        data     = {'url': blob_url}
        response = requests.post(ocr_url, headers=self.__headers, params=params, json=data)
        response.raise_for_status()
        analysis = response.json()
        line_infos = [region["lines"] for region in analysis["regions"]]
        word_infos = self.extract_boundings(line_infos)
        if draw_image:
            self.draw_bounding_boxes(word_infos, blob_url, filename)
        text = self.extract_text_from_ocr(line_infos)
        return " ".join(text), word_infos
    
    def extract_boundings(self, line_infos):
        word_infos = []
        for line in line_infos:
            for word_metadata in line:
                for word_info in word_metadata["words"]:
                    word_infos.append(word_info)
        return word_infos
    
    def extract_text_from_ocr(self, line_infos):
        text = []
        for line in line_infos:
            for word_metadata in line:
                for i in range(len(word_metadata["words"])):
                    text.append(word_metadata["words"][i]['text'])
        return text
    
    def draw_bounding_boxes(self, word_infos, blob_url, filename):
        name = os.path.basename(filename)
        plt.figure(figsize=(50,50))
        image = im = Image.open(requests.get(blob_url, stream=True).raw)
        ax = plt.imshow(image, alpha=0.5)
        for word in word_infos:
            bbox = [int(num) for num in word["boundingBox"].split(",")]
            text = word["text"]
            origin = (bbox[0], bbox[1])
            patch  = Rectangle(origin, bbox[2], bbox[3], fill=False, linewidth=2, color='y')
            ax.axes.add_patch(patch)
            plt.text(origin[0], origin[1], text, fontsize=20, weight="bold", va="top")
        _ = plt.axis("off")
        plt.show()

    
    def __truncatechar(self, text,up_to= 5100):
        _, text = os.path.split(text)
        text = (text[:up_to] + '..') if len(text) > 5100 else text
        return text
                            
    def detect_language(self, text):
        language_api_url = self.__text_anaytics_url + "languages"
        text = self.__truncatechar(text)
        document = { 'documents': [
            { 'id': '1', 'text': text }
        ]}
        response  = requests.post(language_api_url, headers=self.__headers, json=document)
        languages = response.json()
        language = languages['documents'][0]['detectedLanguages'][0]["iso6391Name"]
        return language
    
    def translate(self, language, text,key,translate_to="es"):
        sentiment_api_url = self.__translate_url
        params={"from":language,"to":translate_to}
        text = self.__truncatechar(text,up_to=4900)
        document = [{"Text": text}]
        headers = dict(self.__headers)
        headers['Ocp-Apim-Subscription-Key'] = key
        response  = requests.post(sentiment_api_url, headers=headers, json=document,params=params)
        translations = response.json()
        return translations
    
    def get_entities(self, language, text):
        text = self.__truncatechar(text,up_to=5000)
        key_phrase_api_url =  self.__text_anaytics_url + "entities"
        document = {'documents' : [{'id': '1', 'language': language, 'text': text}]}
        response  = requests.post(key_phrase_api_url, headers=self.__headers, json=document)
        key_phrases = response.json()
        return key_phrases

In [ ]:
blob_base_url = "https://eatraining3939401398.blob.core.windows.net/newspapers/"
image_list =["Boston1.jpg","Boston2.jpg","Boston3.jpg","NewYorkTimes1.jpg","NewYorkTimes2.jpg","NewYorkTimes3.jpg"]

In [ ]:
from azure.storage.blob import BlockBlobService
import json

cs = CognitiveServices(cs_base_url,cs_key,translate_url)     

#Boston newspapers
for image in image_list[:1]:
    (filename, ext) = os.path.splitext(image)
    vision = cs.computer_vision_api(f"{blob_base_url}{image}")
    text, word_infos = cs.ocr(f"{blob_base_url}{image}", image, draw_image=True)
    language = cs.detect_language(text)
    translations = cs.translate(language, text,translate_key)
    entities = cs.get_entities(language,text)
    enrich_data = {}
    enrich_data["Vision"] = vision
    enrich_data["OCR"] = word_infos
    enrich_data["OCRText"] = text
    enrich_data["Language"] = {"language":language}
    enrich_data["Translation"] = translations
    enrich_data["Entities"] = entities

    with open('data.json', 'w',encoding='utf-8') as f:
        json.dump(enrich_data, f)

